## Setup

> Generates the following in database `mosaic_spatial_knn`: (1) table `building_50k`, (2) table `trip_1m`. These are sufficient samples of the full data for this example. __Note:__ You will need to run the actual Spatial KNN on [Databricks ML Runtime](https://docs.databricks.com/en/release-notes/runtime/index.html), for this one it doesn't matter.

<p/>

1. To use Databricks Labs [Mosaic](https://databrickslabs.github.io/mosaic/index.html) library for geospatial data engineering, analysis, and visualization functionality:
  * Install with `%pip install databricks-mosaic`
  * Import and use with the following:
  ```
  import mosaic as mos
  mos.enable_mosaic(spark, dbutils)
  ```
<p/>

2. To use [KeplerGl](https://kepler.gl/) OSS library for map layer rendering:
  * Already installed with Mosaic, use `%%mosaic_kepler` magic [[Mosaic Docs](https://databrickslabs.github.io/mosaic/usage/kepler.html)]
  * Import with `from keplergl import KeplerGl` to use directly

If you have trouble with Volume access:

* For Mosaic 0.3 series (< DBR 13)     - you can copy resources to DBFS as a workaround
* For Mosaic 0.4 series (DBR 13.3 LTS) - you will need to either copy resources to DBFS or setup for Unity Catalog + Shared Access which will involve your workspace admin. Instructions, as updated, will be [here](https://databrickslabs.github.io/mosaic/usage/install-gdal.html).

---
__Last Updated:__ 27 NOV 2023 [Mosaic 0.3.12]

In [0]:
%pip install "databricks-mosaic<0.4,>=0.3" --quiet # <- Mosaic 0.3 series
# %pip install "databricks-mosaic<0.5,>=0.4" --quiet # <- Mosaic 0.4 series (as available)

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# -- configure AQE for more compute heavy operations
#  - choose option-1 or option-2 below, essential for REPARTITION!
# spark.conf.set("spark.databricks.optimizer.adaptive.enabled", False) # <- option-1: turn off completely for full control
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False) # <- option-2: just tweak partition management
spark.conf.set("spark.sql.shuffle.partitions", 1_024)                  # <-- default is 200

# -- import databricks + spark functions
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *

# -- setup mosaic
import mosaic as mos

mos.enable_mosaic(spark, dbutils)
# mos.enable_gdal(spark) # <- not needed for this example

# --other imports
import os
import warnings

warnings.simplefilter("ignore")

__Setup Data Location__

> You can alter this, of course, to match your preferred location. __Note:__ this is showing DBFS for continuity outside Unity Catalog + Shared Access clusters, but you can easily modify paths to use [Volumes](https://docs.databricks.com/en/sql/language-manual/sql-ref-volumes.html), see more details [here](https://databrickslabs.github.io/mosaic/usage/installation.html) as available.

In [0]:
user_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

raw_path = f"dbfs:/{user_name}/geospatial/mosaic/data/spatial_knn"
raw_fuse_path = raw_path.replace("dbfs:","/dbfs")
dbutils.fs.mkdirs(raw_path)

os.environ['RAW_PATH'] = raw_path
os.environ['RAW_FUSE_PATH'] = raw_fuse_path

print(f"The raw data will be stored in '{raw_path}'")

The raw data will be stored in 'dbfs:/mjohns@databricks.com/geospatial/mosaic/data/spatial_knn'


In [0]:
building_filename = "nyc_building_footprints.geojson"
os.environ['BUILDING_FILENAME'] = building_filename

__Setup Catalog and Schema__

> You will have to adjust for your environment.

In [0]:
catalog_name = "mjohns"
sql(f"USE CATALOG {catalog_name}")

db_name = "mosaic_spatial_knn"
sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
sql(f"USE SCHEMA {db_name}")

Out[1]: DataFrame[]

In [0]:
%sql show tables

database,tableName,isTemporary


## Setup NYC Building Data (`Building` Table | 50K)

> While the overall data size is ~1.1M, we are going to just take 50K for purposes of this example.

__Download Data (789MB)__

In [0]:
import requests
import pathlib

def download_url(data_location, dataset_subpath, url):
  fuse_dir = pathlib.Path(data_location.replace('dbfs:',''))
  if (
    not fuse_dir.name.startswith('/Volumes/') and 
    not fuse_dir.name.startswith('/Workspace/')
  ):
    fuse_dir = pathlib.Path(data_location.replace('dbfs:/', '/dbfs/'))
  fuse_dir.mkdir(parents=True, exist_ok=True)
  fuse_path = fuse_dir / dataset_subpath
  if not fuse_path.exists():
    req = requests.get(url)
    with open(fuse_path, 'wb') as f:
      f.write(req.content)
  else:
    print(f"'{fuse_path}' exists...skipping")

In [0]:
# buildings - data preview = https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh
download_url(raw_path, building_filename, "https://data.cityofnewyork.us/api/geospatial/nqwf-w8eh?method=export&format=GeoJSON")

'/dbfs/mjohns@databricks.com/geospatial/mosaic/data/spatial_knn/nyc_building_footprints.geojson' exists...skipping


In [0]:
ls -lh $RAW_FUSE_PATH/$BUILDING_FILENAME

-rwxrwxrwx 1 root root 836M Nov 27 16:45 '/dbfs/mjohns@databricks.com/geospatial/mosaic/data/spatial_knn/nyc_building_footprints.geojson'*


__Generate DataFrame__

In [0]:
@udf(returnType=StringType())
def fix_geojson(gj_dict):
  """
  This GeoJSON has coordinates nested as a string, 
  so standardize here to avoid issues, gets to same as
  expected when `to_json("feature.geometry")` is
  normally called.
  """
  import json
  
  r_list = []
  for l in gj_dict['coordinates']:
    if isinstance(l,str):
      r_list.append(json.loads(l))
    else:
      r_list.append(l)
  
  return json.dumps(
    {
      "type": gj_dict['type'],
      "coordinates": r_list
    }
  )

In [0]:
spark.catalog.clearCache() # <- cache useful for dev (avoid recomputes)

_df_geojson_raw = (
  spark.read
    .option("multiline", "true")
    .format("json")
    .load(f"{raw_path}/{building_filename}")
      .select("type", F.explode(col("features")).alias("feature"))
      .repartition(24)
        .select(
          "type", 
          "feature.properties", 
          fix_geojson("feature.geometry").alias("json_geometry")
        )
    .cache()
)

print(f"count? {_df_geojson_raw.count():,}")
display(_df_geojson_raw.limit(1))

count? 1,109,072


type,properties,json_geometry
FeatureCollection,"List(2042760052, 2048658, 1935, 61809, 2100, Photogramm, {3DCF27FF-A2D0-49BC-A96A-8A25FEEFB8EE}, 78, 40.72, 2017-08-22T00:00:00.000, Constructed, 2042760052, null, 0.0, 0.0)","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-73.85143689311231, 40.85381546242524], [-73.85140609192288, 40.8537759883946], [-73.85145374874513, 40.85375454871117], [-73.8514285730422, 40.853722286769], [-73.85153533063556, 40.85367425956854], [-73.85154239770608, 40.85367108001943], [-73.85157483915371, 40.85371265607289], [-73.85158935911977, 40.853706123670676], [-73.85161245356112, 40.853735721045325], [-73.85159921333282, 40.853741677881], [-73.85160609708538, 40.85375049985463], [-73.8515494571809, 40.85377598115823], [-73.8514909949624, 40.85380228223765], [-73.85148454995291, 40.85379402272926], [-73.85143689311231, 40.85381546242524]]]]}"


In [0]:
_df_geojson = (
  _df_geojson_raw
    .withColumn("geom", mos.st_geomfromgeojson("json_geometry"))
    .withColumn("geom_wkt", mos.st_astext("geom"))
    .withColumn("is_valid", mos.st_isvalid("geom_wkt"))
    .select("properties.*", "geom_wkt", "is_valid")
)

# print(f"count? {_df_geojson.count():,}")
# display(_df_geojson.limit(1))

__Get Sample of 50K__

In [0]:
_df_geojson_50k = (
  _df_geojson
    .sample(0.05)
    .limit(50_000)
)

print(f"count? {_df_geojson_50k.count():,}")

count? 50,000


__Write out to Delta Lake__

In [0]:
(
  _df_geojson_50k
    .write
      .format("delta")
      .mode("overwrite")
      .saveAsTable(f"building_50k")
)

In [0]:
%sql select format_number(count(1), 0) as count from building_50k

count
"50,000"


In [0]:
%sql select * from building_50k limit 1

base_bbl,bin,cnstrct_yr,doitt_id,feat_code,geomsource,globalid,groundelev,heightroof,lstmoddate,lststatype,mpluto_bbl,name,shape_area,shape_len,geom_wkt,is_valid
4032080004,4574701,1925,1201634,5110,Photogramm,{0493656C-85A1-4943-9061-281C9FEB33BA},64,15.04,2017-08-17T00:00:00.000,Constructed,4032080004,null,0.0,0.0,"MULTIPOLYGON (((-73.8545691983773 40.71240691824138, -73.85450439415116 40.71238320422216, -73.8545268433918 40.712347683085575, -73.85459854980775 40.712373923641536, -73.85457857898825 40.71240552349207, -73.85457167676932 40.712402996941606, -73.8545691983773 40.71240691824138)))",true


## Setup NYC Taxi Data (`taxi_trip` | 1M)

> This data is available as part of `databricks-datasets` for customer. We are just going to take 1M trips for our purposes.

__Will write sample out to Delta Lake__

In [0]:
(
  spark.table("delta.`/databricks-datasets/nyctaxi/tables/nyctaxi_yellow`")
    .sample(0.001)
  .withColumn(
    "pickup_point", mos.st_aswkt(mos.st_point(F.col("pickup_longitude"), F.col("pickup_latitude")))
  )
  .withColumn(
    "dropoff_point", mos.st_aswkt(mos.st_point(F.col("dropoff_longitude"), F.col("dropoff_latitude")))
  )
  .limit(1_000_000)
  .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"taxi_trip_1m")
)

In [0]:
%sql select format_number(count(1), 0) as count from taxi_trip_1m

count
"1,000,000"


In [0]:
%sql select * from taxi_trip_1m limit 5

vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code_id,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,pickup_point,dropoff_point
VTS,2009-11-29T03:24:00.000+0000,2009-11-29T03:39:00.000+0000,1,5.2,-73.988922,40.722,null,null,-73.950422,40.7836,CASH,14.1,0.5,0.5,0.0,0.0,15.1,POINT (-73.988922 40.722),POINT (-73.950422 40.7836)
VTS,2009-11-15T01:03:00.000+0000,2009-11-15T01:14:00.000+0000,1,2.75,-74.008792,40.708683,null,null,-73.990708,40.732917,CASH,8.9,0.5,0.5,0.0,0.0,9.9,POINT (-74.008792 40.708683),POINT (-73.990708 40.732917)
VTS,2009-11-18T18:44:00.000+0000,2009-11-18T19:04:00.000+0000,1,3.81,-74.009375,40.712577,null,null,-73.981435,40.760865,CASH,12.5,1.0,0.5,0.0,0.0,14.0,POINT (-74.009375 40.712577),POINT (-73.981435 40.760865)
CMT,2009-11-04T22:53:38.000+0000,2009-11-04T23:04:20.000+0000,1,2.9,-73.997424,40.721479,null,0,-73.974953,40.758131,Cash,9.3,0.5,0.5,0.0,0.0,10.3,POINT (-73.997424 40.721479),POINT (-73.974953 40.758131)
CMT,2009-11-29T00:52:18.000+0000,2009-11-29T01:05:12.000+0000,1,3.9,-73.99881,40.734645,null,0,-73.987929,40.779451,Cash,11.3,0.5,0.5,0.0,0.0,12.3,POINT (-73.99881 40.734645),POINT (-73.987929 40.779451)


## Verify

In [0]:
%sql show tables from mosaic_spatial_knn

database,tableName,isTemporary
mosaic_spatial_knn,building_50k,false
mosaic_spatial_knn,taxi_trip_1m,false


In [0]:
%sql 
-- notice this is a managed table (see 'Location' col_name)
describe table extended building_50k

col_name,data_type,comment
base_bbl,string,null
bin,string,null
cnstrct_yr,string,null
doitt_id,string,null
feat_code,string,null
geomsource,string,null
globalid,string,null
groundelev,string,null
heightroof,string,null
lstmoddate,string,null


In [0]:
%sql 
-- notice this is a managed table (see 'Location' col_name)
describe table extended taxi_trip_1m

col_name,data_type,comment
vendor_id,string,null
pickup_datetime,timestamp,null
dropoff_datetime,timestamp,null
passenger_count,int,null
trip_distance,double,null
pickup_longitude,double,null
pickup_latitude,double,null
rate_code_id,int,null
store_and_fwd_flag,string,null
dropoff_longitude,double,null


## Optional: Clean up initial GeoJSON

> Now that the building data (sample) is in Delta Lake, we don't need it.

In [0]:
display(dbutils.fs.ls(raw_path))

path,name,size,modificationTime
dbfs:/mjohns@databricks.com/geospatial/mosaic/data/spatial_knn/nyc_building_footprints.geojson,nyc_building_footprints.geojson,875673536,1701103503000


In [0]:
# -- uncomment to remove geojson file --
# dbutils.fs.rm(f"{raw_path}/{building_filename}")